In [1]:
# mount google drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
import re
!pip install mojimoji
import mojimoji

!pip install sentencepiece

import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Input, dot, Activation, Bidirectional, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras import backend as K
import tensorflow.keras

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
import io
from nltk.tokenize.toktok import ToktokTokenizer
import unicodedata
import sentencepiece as spm


# ignore warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(tf.__version__)
# gpu
tf.test.is_gpu_available() 

     |████████████████████████████████| 133kB 4.9MB/s 
     |████████████████████████████████| 1.0MB 4.5MB/s 
2.2.0-rc2
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

# load text file

this dataset is aleady implemented a SentenceSpace

In [0]:
num_example = 50000

# create each languages list
def create_lang_list(num_example):
    # load txt file
    lines =  io.open("/content/drive/My Drive/Colab Notebooks/raw.txt", encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_example]]

    return zip(*word_pairs)

In [0]:
en, ja = create_lang_list(num_example)

In [0]:
ja_sentence = list()
for i in ja:
    ja_sentence.append(i.replace(" ", ""))

In [0]:
ja_text = list()
sp = spm.SentencePieceProcessor()
sp.Load("/content/drive/My Drive/wiki-ja.model")
for text in ja_sentence:
    ja_text.append(" ".join(sp.EncodeAsPieces(text)).replace("▁", "").strip())

# Text Pre-processing

 **Removing accented characters**
 e.g. é → e.
 
 **Expanding Contractions**
e.g. don't → do not, I'd → I would


**remove special word**
e.g. remove "123#@!"


**Stemming**
e.g. corder, codes → code


**Lemmatization**
e.g. better → good

**normalize japanese**
e.g.  カナダ  → ｶﾅﾀﾞ

**Tokenize**
 e.g. I am going to restaurant → [[I], [am], [going], [to], [restaurant]]


In [0]:
def moji(text):
  return mojimoji.zen_to_han(text)

# Removing accented characters

English might have accent like é but Japanese doesn't have any accent
I just create different function to ascii for Japanese and English

In [0]:
# Removing accented characters
def english_unicode_to_ascii(text):
     return ''.join(ascii_text for ascii_text in unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore'))

def japanese_unicode_to_ascii(text):
    return ''.join(ascii_text for ascii_text in unicodedata.normalize('NFKD', text))

In [9]:
# e.g.
japanese_unicode_to_ascii("こんにちは。 今日は"), english_unicode_to_ascii("Hello world é ")

('こんにちは。 今日は', 'Hello world e ')

# Expanding Contractions

Japanese doesn't have a Contraction words so I just create a one function to expand Contractions for Engish

# remove special characters and create space between word and punctuation

replacing everything with space except(a-z, A-Z, ".",  "?",  "!",  ",", "-", "ー", , "。", "Kanji", "Katakana", "Hiragana")
create space between word and punctuation (? ! . , 、 。)

In [0]:
def replace_special_character_to_space_en(text):
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)

    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)

    text = text.strip()

    return text
  
def replace_special_character_to_space(text):
    text = re.sub(r"([?!。、¿])", r" \1", text)
    pattern = r"[^\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F?!\s、。.,0-9]+"
    text = re.sub(pattern, '', text).rstrip().strip()
    text = re.sub(r'[" "]+', " ", text)
    text = text.replace("・" , "")

    text = text.lower()

    return text

In [11]:
# e.g.
replace_special_character_to_space("hello #@…123world."), replace_special_character_to_space("こん・にちは・いい天気。")

('123.', 'こんにちはいい天気 。')

# Stemming and Lemmatization

I will do stemming only english which can create a base form of a word from a given word.
Japanese language doesn't need a stemming

In [0]:
def stemmer_word(text):
    ps = nltk.stem.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [13]:
# e.g.
stemmer_word("hello world she has cat but he had dogs he is went to traveling")

'hello world she ha cat but he had dog he is went to travel'

# Text normalize

I will normalize English text and Japanese text using function which is defined so far

In [0]:
def normalize_english(english_text, japanese_text):
    
    input_value = []
    target_value = []
    
    for en_text, ja_text in zip(english_text, japanese_text):
        
        # normalize English
        en_text = english_unicode_to_ascii(en_text)
        en_text = expand_constraction(en_text)
        en_text = replace_special_character_to_space_en(en_text)

        en_text = "start_ " + en_text + " _end"
        # input value doesn't need  a START and END sentence  
        input_value.append(en_text)

        # normalize Japanese
        ja_text = japanese_unicode_to_ascii(ja_text)
        ja_text = replace_special_character_to_space(ja_text)
        ja_text = moji(ja_text)

        # add StTART and END sentence
        ja_text = "start_ " + ja_text + " _end"
        
        target_value.append(ja_text)

    return input_value, target_value

In [0]:
# get normalize text data
input_value, target_value = normalize_english(en, ja_text)

# convert to Series
x = pd.Series(input_value) 
y = pd.Series(target_value)

In [16]:
pd.DataFrame({"input": x, "target": y}).head(20)

,input,target
0,"start_ you are back , aren t you , harold ? _end",start_ あなた は 戻った の ね ﾊﾛﾙﾄ゙ ? _end
1,start_ my opponent is shark . _end,start_ 俺 の 相手 は ｼｬｰ ｸ だ ｡ _end
2,start_ this is one thing in exchange for anoth...,start_ 引き 換え だ ある 事 とある 物の _end
3,"start_ yeah , i m fine . _end",start_ もう いい よ ご ち そう さま う うん _end
4,start_ don t come to the office anymore . don ...,start_ もう 会社 には 来 ない で くれ 電話 も する な _end
5,start_ looks beautiful . _end,start_ きれい だ ｡ _end
6,"start_ get him out of here , because i will fu...",start_ 連れ て 行 け 殺し そう だ わ かった か ? _end
7,start_ you killed him ! _end,start_ 殺 した のか ! _end
8,"start_ okay , then who ? _end",start_ わ ぁ ! いつも すみ ません ｡ いい の よ ｡ _end
9,start_ it seems a former employee . . . _end,start_ ｶﾝﾊ゚ﾆｰ の元 社員 が _end


# tokenize
tokenize each language word based on space

In [0]:
def tokenize(lang):
    # vectorize a text corpus
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters=' ')

    # updates internal vocabulary based on a list of texts
    # e.g. "[this place is good ]"→{this:2, place:3, is:1, good:4} "
    lang_tokenizer.fit_on_texts(lang)

    # Transforms each text in texts to a sequence of integers.
    # e.g. this place is good → [[2, 3, 1, 4]]
    tensor = lang_tokenizer.texts_to_sequences(lang)

    # transform a list of num sample into a 2D Numpy array of shape 
    # Fixed length because length of sequence of integers are different
    # return (len(sequences), maxlen)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                          padding='post')
    return tensor, lang_tokenizer

In [18]:
# e.g.
tokenize(['this place is good', "こんにちは 今日は いい天気 。", "today is so cold"])

(array([[ 2,  3,  1,  4],
        [ 5,  6,  7,  8],
        [ 9,  1, 10, 11]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x7f408187a860>)

# create clean dataset

In [0]:
# cleate a clean dataset
def create_dataset(en, ja):
    
    # input_tensor, target_tensor: 2d numpy array
    # input_lang_tokenize, target_lang_tokenize: word dictionary
    input_tensor, input_lang_tokenize = tokenize(en)
    target_tensor, target_lang_tokenize = tokenize(ja)

    return input_tensor, target_tensor, input_lang_tokenize, target_lang_tokenize

In [0]:
input_tensor, target_tensor, input_lang_tokenize, target_lang_tokenize = create_dataset(x, y)

In [21]:
target_lang_tokenize

In [0]:
def max_length(input_tensor, target_tensor):


    # max length of input sentense and target sentense
    english_len = [len(i) for i in input_tensor]

    japanese_len = [len(i) for i in target_tensor]

 
    # print max length
    print("english length:", max(english_len))
    print("japanese length:", max(japanese_len))
    max_len_input =  max(english_len)
    max_len_target =  max(japanese_len)

    return max_len_input, max_len_target

In [140]:
# Calculate max_length of the target tensors
max_length_input, max_length_target = max_length(input_tensor, target_tensor)

english length: 66
japanese length: 43


In [25]:
# create trainnig set and validation set
input_tensor_train, input_tensor_val, \
    target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=42)

# show length
print(len(input_tensor_train), len(input_tensor_val), len(target_tensor_train), len(target_tensor_val))

40000 10000 40000 10000


In [0]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            # Index number assigned to each word
            print("%d----->%s" % (t, lang.index_word[t]))

In [27]:
print("input lang: index to word mapping")
convert(input_lang_tokenize, input_tensor_train[10])
print("output lang: index to word mapping")
convert(target_lang_tokenize, target_tensor_train[10])

input lang: index to word mapping
1----->start_
2871----->serial
4192----->killers
3----->.
2----->_end
output lang: index to word mapping
1----->start_
931----->研究
3177----->分野
4----->は
7754----->ｼﾘｱﾙ
4362----->ｷﾗｰ
10----->だ
2----->_end


# define parameter

In [28]:
# BUFFER_SIZE >= dataset if smaller than dataset can't shuffle equally
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
dropout_rate = 0.3
# if None steps_per_epoch == mum of dataset
train_steps_per_epoch = len(input_tensor_train)
val_steps_per_epoch = len(input_tensor_val)
embedding_dim = 300
units = 1024
vocab_inp_size = len(input_lang_tokenize.word_index) + 1
print('Total unique words in the input: %s' % len(input_lang_tokenize.word_index))
vocab_tar_size = len(target_lang_tokenize.word_index) + 1


# create train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# create validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).batch(BATCH_SIZE, drop_remainder=True)

Total unique words in the input: 21576


In [29]:
example_input_batch, example_target_batch =  next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 66]), TensorShape([64, 43]))

**explore japanese Word2Vec pre-train model**

# Encoder Model

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size, dropout_rate):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.dropout = Dropout(dropout_rate)
        self.embedding = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim)
        self.first_lstm = tf.keras.layers.LSTM(self.enc_units,
                                                            return_sequences=True,
                                                            recurrent_initializer='glorot_uniform')
        
        self.final_lstm = tf.keras.layers.LSTM(self.enc_units,
                                                    return_sequences=True,
                                                    return_state=True,
                                                    recurrent_initializer='glorot_uniform')
        
    def call(self, x, hidden):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.first_lstm(x, initial_state =hidden)
        output, state_h, state_c = self.final_lstm(x)
        state = [state_h, state_c ]

        return output, state
        
    def initialize_hidden_state(self):
            return tf.zeros((self.batch_size , self.enc_units)), tf.zeros((self.batch_size , self.enc_units))

# attention

In [0]:
class SimpleAttention(tf.keras.models.Model):

    def __init__(self, units: int, *args, **kwargs):

        super().__init__(*args, **kwargs)
        self.units = units

        self.q_dense_layer = Dense(units, use_bias=False, name='q_dense_layer')
        self.k_dense_layer = Dense(units, use_bias=False, name='k_dense_layer')
        self.v_dense_layer = Dense(units, use_bias=False, name='v_dense_layer')
        self.output_dense_layer = Dense(units, use_bias=False, name='output_dense_layer')

    def call(self, input, memory):

        q = self.q_dense_layer(input) 
        k = self.k_dense_layer(memory) 
        v = self.v_dense_layer(memory)

        depth = self.units // 2
        q *= depth ** -0.5  # for scaled dot production

        # caluclate relation between query and key
        logit = tf.matmul(q, k, transpose_b=True) 

        attention_weight = tf.nn.softmax(logit)

        attention_output = tf.matmul(attention_weight, v) 
        return self.output_dense_layer(attention_output)

# Decoder Model

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size, target_embedding_matrix, dropout_rate):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_tar_size, embedding_dim)
        self.dropout = Dropout(dropout_rate)
        self.first_lstm = tf.keras.layers.LSTM(self.dec_units,
                                                            return_sequences=True)
        self.final_lstm = tf.keras.layers.LSTM(self.dec_units,
                                                            return_sequences=True,
                                                            return_state=True)
                                                            
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        self.attention = SimpleAttention(self.dec_units)
    
    def call(self, x, hidden, enc_output):
        x = self.embedding(x)
        x = self.dropout(x)
        
        x =  self.first_lstm(x)
        output, state_h, state_c = self.final_lstm(x)
        state = [state_h, state_c]
        attention_weights = self.attention(output, enc_output)
        output = tf.concat([output, attention_weights], axis=-1)

                
        output = tf.reshape(output, (-1, output.shape[2]))
        
        output = self.fc(output)
        
        return  output, state, attention_weights

# optimizer and the loss function

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, epsilon=1e-04, decay=1e-06)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

# Checkpoints

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

# train model

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([target_lang_tokenize.word_index['start_']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [41]:
EPOCHS = 20

for epoch in range(EPOCHS):
  

  enc_hidden = encoder.initialize_hidden_state()
  train_loss = 0
  val_loss = 0

  for (batch, (inp, targ)) in enumerate(train_dataset.take(train_steps_per_epoch)):
    train_batch_loss = train_step(inp, targ, enc_hidden)
    train_loss += train_batch_loss

    if batch % 200 == 0:
      print('Train datta Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   train_batch_loss.numpy()))

  for (batch, (val_inp, val_tar)) in enumerate(val_dataset.take(val_steps_per_epoch)):
    val_batch_loss = train_step(val_inp, val_tar, enc_hidden)
    val_loss += val_batch_loss

    if batch % 200 == 0:
      print('validation datta Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   val_batch_loss.numpy()))


  # saving (checkpoint) the model every 2 epochs
  # if (epoch + 1) % 2 == 0:
  #   checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      train_loss / train_steps_per_epoch))
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      val_loss / val_steps_per_epoch))


Train datta Epoch 1 Batch 0 Loss 1.0631
Train datta Epoch 1 Batch 200 Loss 1.1212
Train datta Epoch 1 Batch 400 Loss 1.0413
Train datta Epoch 1 Batch 600 Loss 1.1313
validation datta Epoch 1 Batch 0 Loss 1.1407
Epoch 1 Loss 0.0179
Epoch 1 Loss 0.0175
Train datta Epoch 2 Batch 0 Loss 1.0392
Train datta Epoch 2 Batch 200 Loss 1.1046
Train datta Epoch 2 Batch 400 Loss 1.1102
Train datta Epoch 2 Batch 600 Loss 1.0447
validation datta Epoch 2 Batch 0 Loss 1.1103
Epoch 2 Loss 0.0172
Epoch 2 Loss 0.0167
Train datta Epoch 3 Batch 0 Loss 1.0580
Train datta Epoch 3 Batch 200 Loss 1.1142
Train datta Epoch 3 Batch 400 Loss 1.0408
Train datta Epoch 3 Batch 600 Loss 1.0748
validation datta Epoch 3 Batch 0 Loss 1.0770
Epoch 3 Loss 0.0166
Epoch 3 Loss 0.0161
Train datta Epoch 4 Batch 0 Loss 1.0393
Train datta Epoch 4 Batch 200 Loss 1.1169
Train datta Epoch 4 Batch 400 Loss 1.1401
Train datta Epoch 4 Batch 600 Loss 1.0088
validation datta Epoch 4 Batch 0 Loss 1.0497
Epoch 4 Loss 0.0161
Epoch 4 Loss 0.0

# Evaluation

In [0]:
def preprocess_sentence(en_text):
        # normalize English
        en_text = en_text.lower()
        en_text = english_unicode_to_ascii(en_text)
        # en_text = expand_constraction(en_text)
        en_text = replace_special_character_to_space_en(en_text)


        en_text = "start_ " + en_text + " _end"
        return en_text
        

def evaluate(sentence):
  
    attention_plot = np.zeros((max_length_target, max_length_input))

    sentence = preprocess_sentence(sentence)
    inputs = [input_lang_tokenize.word_index[i] for i in sentence.split(' ')]

    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_input,
                                                           padding='post')
    
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
    enc_out, state = encoder(inputs, hidden)
    hidden_state = state
    dec_input = tf.expand_dims([target_lang_tokenize.word_index['start_']], 0)
    for t in range(max_length_target):
        predictions, hidden_state, _ = decoder(dec_input,
                                                             hidden_state,
                                                             enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()

        result += target_lang_tokenize.index_word[predicted_id] + ' '

        if target_lang_tokenize.index_word[predicted_id] == '_end' or len(result) > max_length_target:
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence, 

In [0]:
from nltk.translate.bleu_score import sentence_bleu

def result(sentence):
    result, sentence, _= evaluate(sentence)

    return result, sentence

In [115]:
    result, sentence = result("he has a dog")
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    print('BLEU corpus: %f' % sentence_bleu(["彼は犬を飼っている"], result))

Input: start_ he has a dog _end
Predicted translation: 彼は 犬 か _end 
BLEU corpus: 0.388273


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [118]:
result, sentence = result("What is this!?")
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))
print('BLEU corpus: %f' % sentence_bleu(["なにこれ !?"], result))

Input: start_ what is this ! ? _end
Predicted translation: 何 ? _end 
BLEU corpus: 0.686589


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [110]:
result, sentence = result("Actually I have it.")
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))
print('BLEU corpus: %f' % sentence_bleu(["実は持っている"], result))

Input: start_ actually i have it . _end
Predicted translation: 実は 私 には _end 
BLEU corpus: 0.336493


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [112]:
result, sentence = result("she is beautiful")
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))
print('BLEU corpus: %f' % sentence_bleu(["彼女は美しい"], result))

Input: start_ she is beautiful _end
Predicted translation: この 美しい _end 
BLEU corpus: 0.259654


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
